In [1]:
import os

In [2]:
%pwd

'/workspaces/-renci_Perfonmans_Tahminlemesi_MLOps_Deneme2/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/workspaces/-renci_Perfonmans_Tahminlemesi_MLOps_Deneme2'

# Entity(Varliklar)

degiskenlerimi ve degiskenlerin tiplerini olusturucam

In [5]:
from dataclasses import dataclass

In [6]:
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

# Configrasyon

Yukarida degiskenlerini tanimladigimiz yapilarin degerlerini vericegiz.

In [7]:
from src.ÖğrenciTahminleme.constants import * # constanst icerisinde degiskenleri import ettik
from src.ÖğrenciTahminleme.utils.common import read_yaml, create_directories # common.py icerisinde read_yaml ve create_directories methodlarini import ettik

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig: # Yukarida DataIngestionConfig sinifi icerisinde tanimlamis oldugum degiskenleri return edecektir
        config = self.config.data_ingestion # root_dir, local_data_file, source_URL, unzip_dir keylerine erisim sagliyorum

        create_directories([config.root_dir]) # artifacts/data_ingestion isimli bir klasor olustuyorum

        data_ingestion_config = DataIngestionConfig( # Ust hucrede tanimlamis oldugum sinifin nesnesini yaratiyorum
            root_dir=config.root_dir, #artifacts/data_ingestion
            source_URL=config.source_URL, #https://www.kaggle.com/api/v1/datasets/download/bhavikjikadara/student-study-performance
            local_data_file=config.local_data_file, # artifacts/data_ingestion/data.zip
            unzip_dir=config.unzip_dir, # artifacts/data_ingestion
        )

        return data_ingestion_config

# Components(Moduller)

Burada yapilacak is ne ise onun adimlari tanimlanir

In [9]:
import os
import urllib.request as request
import zipfile
from src.ÖğrenciTahminleme import logger
from src.ÖğrenciTahminleme.utils.common import get_size
import pandas as pd
from sklearn.model_selection import train_test_split


In [10]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig): # config parametresi DataIngestionConfig icerisnde olusturulmus degiskenlerini ve tiplerini almasi saglaniyor
        self.config = config # config degiskenine deger olarak DataIngestion sinifindan nesne olusturken config parametresinin yerine ne deger ataniyorsa aynisi config degiskenine atanmasi saglaniyor


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file): # data.zip isimli bir dosya yoksa True Donecektir
            filename, headers = request.urlretrieve( # filename degiskenin dosyanin ismi ataniyor headers degiskenine dosyanin bilgileri ataniyor
                url = self.config.source_URL, # url parametre dosyanin hangi siteden indirilmesi gerektigini alir
                filename = self.config.local_data_file # dosyayi hangi isimle locale indirmen gerektigini yazar
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else: # eger onceden boyle bir dosya varsa
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  #data.zip dosyasinin kb olarak ne kadar buyuk oldugunu loglar


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir # unzip_path degiskenine artifacts/data_ingestion bu degeri atiyoruz
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref: # zip_ref adini verdigimiz bir degiskenle data.zipin icerini acip okuyoruz
            zip_ref.extractall(unzip_path) # extractall metoduyla dosyami zipten cikarip artifacts/data_ingestion bu klasor icerisine atiyorum

# Pipeline
Yukarida olusturdugumuz butun siniflari teker teker bir nesne olusturup cagiriyoruz

In [11]:
try:
    config = ConfigurationManager() #benim değişkenlerimin değerlerini tutuyor.
    data_ingestion_config = config.get_data_ingestion_config() # bu methodla config.yaml içindeki bütün değerleri çağırıyorum
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file() #görüntüleri indirir ve zip uzantılı olarak verilen dizine atar
    data_ingestion.extract_zip_file() # zipten çıkarır ve verilen dosya dizinde kayıt eder
except Exception as e:
    raise e

[2025-01-17 22:25:48,177: INFO: 2769941048: artifacts/data_ingestion/data.zip download! with following info: 
X-GUploader-UploadID: AFIdbgS_n5vj1sXTXdIxw6Cs5ILqOA9px3Sgl8hM36OKW6DGLEo8p1suW-nSxTJ8QhNvKZKtpVz4Ous
Expires: Fri, 17 Jan 2025 22:25:48 GMT
Date: Fri, 17 Jan 2025 22:25:48 GMT
Cache-Control: private, max-age=0
Last-Modified: Thu, 07 Mar 2024 06:14:12 GMT
ETag: "47d2ff7dbfc7997cc2cf67cb80e41cdd"
x-goog-generation: 1709792052142033
x-goog-metageneration: 1
x-goog-stored-content-encoding: identity
x-goog-stored-content-length: 8907
Content-Type: application/zip
x-goog-hash: crc32c=A49eQg==
x-goog-hash: md5=R9L/fb/HmXzCz2fLgOQc3Q==
x-goog-storage-class: MULTI_REGIONAL
Accept-Ranges: bytes
Content-Length: 8907
Access-Control-Allow-Origin: *
Access-Control-Expose-Headers: Content-Type
Access-Control-Expose-Headers: User-Agent
Server: UploadServer
Alt-Svc: h3=":443"; ma=2592000,h3-29=":443"; ma=2592000
Connection: close

]
